In [1]:
import sys
sys.path.append(r"C:\Users\PMLS\timeseires")

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
import os
from keras.callbacks import Callback

In [5]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [7]:
def create_lstm():
    input_data = Input(shape=(time_steps, num_features))
    lstm_layer1 = LSTM(8, return_sequences=True)(input_data)
    lstm_layer2 = LSTM(20)(lstm_layer1)
    x = Flatten()(lstm_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [9]:
model1 = create_lstm()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 24, 8)               │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 20)                  │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,301 (12.89 KB)

 Trainable params: 3,301 (12.89 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [13]:
checkpoints = r'C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\PMLS\shahid (ML-lab)\lab10'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
C:\Users\PMLS\AppData\Local\Temp\ipykernel_18420\292278908.py:3: SyntaxWarning: invalid escape sequence '\h'
  FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
C:\Users\PMLS\AppData\Local\Temp\ipykernel_18420\292278908.py:4: SyntaxWarning: invalid escape sequence '\h'
  JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])


In [15]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [17]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_lstm()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [21]:
import os
path_dataset =r'C:\Users\PMLS\shahid (ML-lab)\lab7'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [23]:
time_steps=24
num_features=21

In [25]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.7203304767608643 sec


In [29]:
epochs = 40
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/40
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0309 - mae: 0.0309 - mape: 116.6950
Epoch 1: val_loss improved from inf to 0.02014, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0309 - mae: 0.0309 - mape: 116.7009 - val_loss: 0.0201 - val_mae: 0.0201 - val_mape: 9.9500
Epoch 2/40
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0174 - mae: 0.0174 - mape: 329.3855
Epoch 2: val_loss improved from 0.02014 to 0.01253, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0174 - mae: 0.0174 - mape: 329.4239 - val_loss: 0.0125 - val_mae: 0.0125 - val_mape: 5.1421
Epoch 3/40
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0127 - mae: 0.0127 - mape: 17.7926
Epoch 3: val_loss improved from 0.01253 to 0.01174, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0127 - mae: 0.0127 - mape: 17.9814 - val_loss: 0.0117 - val_mae: 0.0117 - val_mape: 6.4150
Epoch 4/40
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0112 - mae: 0.0112 - mape: 191.5311
Epoch 4: val_loss improved from 0.01174 to 0.00883, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0112 - mae: 0.0112 - mape: 191.5152 - val_loss: 0.0088 - val_mae: 0.0088 - val_mape: 3.8447
Epoch 5/40
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0100 - mape: 65.0246
Epoch 5: val_loss did not improve from 0.00883
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0100 - mae: 0.0100 - mape: 65.3228 - val_loss: 0.0090 - val_mae: 0.0090 - val_mape: 3.8286
Epoch 6/40
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0095 - mape: 94.1026
Epoch 6: val_loss did not improve from 0.00883
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0095 - mae: 0.0095 - mape: 94.1979 - val_loss: 0.0107 - val_mae: 0.0107 - val_mape: 4.5251
Epoch 7/40
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0088 - mae: 0.0088 - mape: 18.0083
Epoch 7: val_loss improved from 0.00883 to 0.00798, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0007-loss0.01.h5


...

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0073 - mae: 0.0073 - mape: 22.8506 - val_loss: 0.0068 - val_mae: 0.0068 - val_mape: 2.9628
Epoch 17/40
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0073 - mae: 0.0073 - mape: 4.7204
Epoch 17: val_loss did not improve from 0.00677
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0073 - mae: 0.0073 - mape: 4.7447 - val_loss: 0.0080 - val_mae: 0.0080 - val_mape: 3.6529
Epoch 18/40
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0072 - mae: 0.0072 - mape: 16.6186
Epoch 18: val_loss did not improve from 0.00677
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0072 - mae: 0.0072 - mape: 16.9129 - val_loss: 0.0088 - val_mae: 0.0088 - val_mape: 3.6546
Epoch 19/40
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0072 - mae: 0.0072 - mape: 8.0436
Epoch 19: val_loss did not improve from 0.00677
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0072 - mae: 0.0072 - mape: 8.1820 - val_loss: 0.0071 - val_mae: 0.0071

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - loss: 0.0071 - mae: 0.0071 - mape: 72.3409 - val_loss: 0.0067 - val_mae: 0.0067 - val_mape: 2.8190
Epoch 22/40
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0070 - mae: 0.0070 - mape: 52.8886
Epoch 22: val_loss did not improve from 0.00670
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0070 - mae: 0.0070 - mape: 53.0517 - val_loss: 0.0076 - val_mae: 0.0076 - val_mape: 3.4081
Epoch 23/40
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0070 - mae: 0.0070 - mape: 239.4680
Epoch 23: val_loss improved from 0.00670 to 0.00645, saving model to C:\Users\PMLS\shahid (ML-lab)\lab10\\E1-cp-0023-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - loss: 0.0070 - mae: 0.0070 - mape: 239.2585 - val_loss: 0.0065 - val_mae: 0.0065 - val_mape: 2.7214
Epoch 24/40
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0069 - mae: 0.0069 - mape: 46.8655
Epoch 24: val_loss did not improve from 0.00645
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0069 - mae: 0.0069 - mape: 46.8984 - val_loss: 0.0074 - val_mae: 0.0074 - val_mape: 2.9965
Epoch 25/40
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0069 - mae: 0.0069 - mape: 2.3460
Epoch 25: val_loss did not improve from 0.00645
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0069 - mae: 0.0069 - mape: 2.3718 - val_loss: 0.0070 - val_mae: 0.0070 - val_mape: 3.0729
Epoch 26/40
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0068 - mae: 0.0068 - mape: 2.7928
Epoch 26: val_loss did not improve from 0.00645
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0068 - mae: 0.0068 - mape: 2.9314 - val_loss: 0.0069 - val_mae: 0.006

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0068 - mae: 0.0068 - mape: 96.8506 - val_loss: 0.0063 - val_mae: 0.0063 - val_mape: 2.6150
Epoch 31/40
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0066 - mae: 0.0066 - mape: 68.5967
Epoch 31: val_loss did not improve from 0.00630
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0066 - mae: 0.0066 - mape: 68.6022 - val_loss: 0.0066 - val_mae: 0.0066 - val_mape: 2.6769
Epoch 32/40
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0067 - mae: 0.0067 - mape: 2.3250
Epoch 32: val_loss did not improve from 0.00630
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0067 - mae: 0.0067 - mape: 2.3276 - val_loss: 0.0067 - val_mae: 0.0067 - val_mape: 2.7623
Epoch 33/40
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0066 - mae: 0.0066 - mape: 106.8690
Epoch 33: val_loss did not improve from 0.00630
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - loss: 0.0066 - mae: 0.0066 - mape: 106.8644 - val_loss: 0.0075 - val_mae: 0.

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0066 - mae: 0.0066 - mape: 19.0369 - val_loss: 0.0063 - val_mae: 0.0063 - val_mape: 2.5955
Epoch 36/40
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0065 - mae: 0.0065 - mape: 183.1278
Epoch 36: val_loss did not improve from 0.00626
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0065 - mae: 0.0065 - mape: 182.9810 - val_loss: 0.0078 - val_mae: 0.0078 - val_mape: 3.5293
Epoch 37/40
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0065 - mae: 0.0065 - mape: 86.1404
Epoch 37: val_loss did not improve from 0.00626
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0065 - mae: 0.0065 - mape: 86.2401 - val_loss: 0.0075 - val_mae: 0.0075 - val_mape: 3.2413
Epoch 38/40
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0065 - mae: 0.0065 - mape: 101.6006
Epoch 38: val_loss did not improve from 0.00626
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0065 - mae: 0.0065 - mape: 101.6734 - val_loss: 0.0073 - val_mae

In [31]:

model = load_model(r'C:\Users\PMLS\shahid (ML-lab)\lab10\E1-cp-0035-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Mean Absolute Error (MAE): 102.11
Median Absolute Error (MedAE): 78.59
Mean Squared Error (MSE): 19148.84
Root Mean Squared Error (RMSE): 138.38
Mean Absolute Percentage Error (MAPE): 0.69 %
Median Absolute Percentage Error (MDAPE): 0.54 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [33]:
checkpoints = r'C:\Users\PMLS\shahid (ML-lab)\lab10\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\PMLS\shahid (ML-lab)\lab10\E1-cp-0035-loss0.01.h5'
start_epoch= 34

In [35]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\PMLS\shahid (ML-lab)\lab10\E1-cp-0035-loss0.01.h5...


TypeError: Could not locate function 'mae'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mae', 'registered_name': 'mae'}

In [41]:
import tensorflow as tf
from keras.metrics import mean_absolute_error as mae
from keras.metrics import mean_absolute_percentage_error as mape
from keras.models import load_model
import numpy as np
from keras import backend as K

# Enable eager execution (important for some TensorFlow versions)
tf.config.run_functions_eagerly(True)

# Define your model path (replace with your actual path)
model_path = r'C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5'

# construct the callback to save only the *best* model to disk
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1 = TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1, TrainingMonitor1]

# if there is no specific model checkpoint supplied, initialize new model
if model_path is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mean_absolute_error', 
                 optimizer=opt, 
                 metrics=["mean_absolute_error", "mean_absolute_percentage_error"])
else:
    print("[INFO] loading {}...".format(model_path))
    try:
        # First try loading with custom objects
        model = load_model(model_path, custom_objects={
            'mae': mae,
            'mape': mape,
            'mean_absolute_error': mae,
            'mean_absolute_percentage_error': mape
        })
    except Exception as e:
        print(f"First load attempt failed: {e}")
        try:
            # Fallback to loading without custom objects
            model = load_model(model_path)
        except Exception as e:
            print(f"Second load attempt failed: {e}")
            try:
                # Final fallback - load without compilation
                model = load_model(model_path, compile=False)
                model.compile(loss='mean_absolute_error',
                            optimizer=Adam(1e-4),
                            metrics=[mae, mape])
            except Exception as e:
                print(f"Final load attempt failed: {e}")
                raise RuntimeError("Could not load model with any method")

    # Update learning rate safely
    try:
        if hasattr(model.optimizer, 'learning_rate'):
            old_lr = float(K.get_value(model.optimizer.learning_rate))
            print("[INFO] old learning rate: {}".format(old_lr))
            K.set_value(model.optimizer.learning_rate, np.float32(1e-4))
            new_lr = float(K.get_value(model.optimizer.learning_rate))
            print("[INFO] new learning rate: {}".format(new_lr))
    except Exception as e:
        print(f"[WARNING] Could not update learning rate: {e}")


[INFO] loading C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5...
[WARNING] Could not update learning rate: module 'keras.api.backend' has no attribute 'get_value'


In [43]:
# Convert all data to numpy arrays if they aren't already
train_X = np.array(train_X) if not isinstance(train_X, np.ndarray) else train_X
train_y = np.array(train_y) if not isinstance(train_y, np.ndarray) else train_y
validation_X = np.array(validation_X) if not isinstance(validation_X, np.ndarray) else validation_X
validation_y = np.array(validation_y) if not isinstance(validation_y, np.ndarray) else validation_y


In [53]:
# Training with proper eager execution
epochs = 10
verbose = 1
batch_size = 32
    History = model.fit(train_X,
                      train_y,
                      batch_size=batch_size,   
                      epochs=epochs, 
                      validation_data=(validation_X, validation_y),
                      callbacks=callbacks,
                      verbose=verbose)

    

IndentationError: unexpected indent (2303810755.py, line 5)

In [47]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

C:\Users\PMLS\anaconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10


ValueError: Unknown variable: <Variable path=simple_rnn_2/simple_rnn_cell/kernel, shape=(21, 8), dtype=float32, value=[[-1.28049254e-01  7.60119021e-01 -9.84267831e-01  1.15700948e+00
  -9.11933541e-01  2.83530682e-01 -1.34242678e+00  1.28985071e+00]
 [-1.08757569e-03 -2.87455440e-01 -6.33366778e-02  1.04126260e-01
   4.09385622e-01 -2.35603452e-02  4.21401829e-01 -2.81358153e-01]
 [ 4.82359864e-02 -2.71494329e-01 -8.88475105e-02  6.61750957e-02
   5.46873391e-01 -4.33829501e-02  4.13632691e-01 -2.72765398e-01]
 [-4.53201272e-02  2.03003913e-01  8.30770433e-02 -2.10338145e-01
  -3.79177064e-01 -1.02298096e-01  6.74365237e-02 -1.96737200e-01]
 [ 8.92618224e-02  2.36484662e-01  5.13518415e-02 -2.98869014e-01
   1.14898704e-01 -7.81441256e-02  3.14417779e-02 -1.62379459e-01]
 [-4.81481059e-03  8.59820098e-02 -4.27903719e-02 -1.38831019e-01
  -5.20341396e-02  1.11557193e-01  2.79625151e-02  7.72579312e-02]
 [-4.96157669e-02  7.29206577e-02 -5.24620302e-02 -2.88439959e-01
   1.24558844e-01 

In [19]:

model = load_model(r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab8\E2-cp-0008-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 2ms/step
Mean Absolute Error (MAE): 103.3
Median Absolute Error (MedAE): 82.18
Mean Squared Error (MSE): 18772.44
Root Mean Squared Error (RMSE): 137.01
Mean Absolute Percentage Error (MAPE): 0.71 %
Median Absolute Percentage Error (MDAPE): 0.57 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
